In [1]:
import pandas as pd
import numpy as np 
import re

# Data comes in as daily csv with long term historical data
 
# NOTES:

# 1. Can vary how buy more or less stock. Choose either using a simple bool, like if less than previous month, buy.
#    Or, we can have if less by x amount, buy more by y amount. Harder to formulate...
# 2. How to optimize multiple parameters? At this point I have 4. Length of moving average, money percent increment, 
#    Percent deviation from Moving Average and finally the monthly allocation, more to come I'm sure... matrix? brute force?

In [22]:
def comparetodca(moneyinc,percentdev,monthlyalloc,stockname):
    
    malength = 4 # Moving average length, initially at 4
    moneyincrement = moneyinc # Money increase or decrease from previous timestep
    percentdeviation = percentdev # Percent deviation from allocated monthly
    monthlyallocation = monthlyalloc # Test, $800 per month, can change to annual

    #global numberofstocksheld 
    numberofstocksheld = 0 # Number of securities held
    
    stockprice = 0.0 # Current price of security
        
    data = pd.read_csv(stockname)
    
    # Define parameters of program. Sticking with monthly for now
    stockdate = data['Date']
    stockprice = data['Open'] # Stick with opening price

    movingavgarray = np.zeros(malength)
    movingavgarray[-1] = monthlyallocation

    # Cycle through all the dates, essentially the csv

    # This is to test how we do a simple bool check with the money being less or more than the last cycle (in this case monthly)
    # The moving average is to track our buying power, and increase automatically by 10% until we exceed a tolerance on the moving
    # average. 

    buyingpower = monthlyallocation # Our buying power is set to the monthly alloc parameter above
    laststockprice = 9999 # The first time we run this, it'll hold true so we always increase our buying power. Just a setting
    outoftol = False # Bool to turn on if our buying power went out of tolerance
    stockisint = True # Bool to check if stock is int
    firsttime = True # Bool to check if first set of cycle, ever. 
    totalmoneyspent = 0

    for i in range(len(stockdate)):
    #for i in range(450):
        nonzeromalen = np.count_nonzero(movingavgarray) # Length of moving avg array. Should be same as malength but if start early and issue...
        movingaverage = sum(movingavgarray)/nonzeromalen # Moving average price of buying power. 
        currentdate = stockdate[i]
        currentprice = stockprice[i]
        dailydate = currentdate.split('-') # Split on -
        day = dailydate[2] # Dailydate=[yyyy,mm,dd]

        if day == '15': # Just choosing middle of the month

            # Check before we buy, to buy more that day.
            if (currentprice < laststockprice) and outoftol == False:
                buyingpower=buyingpower*(1+moneyincrement) # Add 10%
                print('Price Increase')
                print(buyingpower)
            if firsttime == True:
                buyingpower = monthlyallocation

            # Cycle, simulates buying the stock
            numberofstocks = buyingpower/currentprice # Number of stocks

            # Is the stock int?
            # If not int...
            if type(numberofstocks) != int:
                # Round down
                numberofstocks = int(numberofstocks)
                #print(numberofstocks)
                buyingpower = numberofstocks*currentprice

            numberofstocksheld += numberofstocks # Just bought. Added
            totalmoneyspent += buyingpower
            
            movingavgarray=np.roll(movingavgarray,-1) # Shift array down 1
            movingavgarray[-1] = buyingpower # Last element (most recent) 
            
            # Check our current buying power vs the moving average. Do something if above/below
            # Greater than 10% of our moving average

            Uppertolerance = monthlyallocation*(1+percentdeviation)
            Lowertolerance = monthlyallocation-(monthlyallocation*percentdeviation)
            
            #print(movingavgarray)
            if buyingpower > Uppertolerance:
                buyingpower = (monthlyallocation*(nonzeromalen+1))-sum(movingavgarray)
                #print("Greater")
                #print(sum(movingavgarray))
                outoftol = True
                #print(buyingpower)

            # Less than 10% of our moving average
            elif buyingpower < Lowertolerance:
                buyingpower = (monthlyallocation*(nonzeromalen+1))-sum(movingavgarray)
                #print("Lesser")
                outoftol = False
                #print(buyingpower)

            laststockprice = currentprice # Update our last stock price for the current one. For next month/cycle  
            firsttime = False

            print(numberofstocksheld)
            print(currentprice)
            print(movingavgarray)
            #print("$Price:")
            #print(currentprice)
            #print("No. of Stock:")
            #print(numberofstocks)

        #buyingpower = newbuyingpower        
    print("My formula")
    print(numberofstocksheld)
    print(totalmoneyspent)

    # Simply dollar monthly cost averaging
    totaldollarcostavgcost = 0
    dca_numberofstocksheld = 0
    for i in range(len(stockdate)):
    #for i in range(450):
        nonzeromalen = np.count_nonzero(movingavgarray) # Length of moving avg array. Should be same as malength but if start early and issue...
        movingaverage = sum(movingavgarray)/nonzeromalen # Moving average price of buying power. 
        currentdate = stockdate[i]
        currentprice = stockprice[i]
        dailydate = currentdate.split('-') # Split on -
        day = dailydate[2] # Dailydate=[yyyy,mm,dd]

        if day == '15': # Just choosing middle of the month

            # Cycle, simulates buying the stock
            numberofstocks = int(monthlyallocation/currentprice) # Number of stocks
            dca_numberofstocksheld += numberofstocks # Just bought. Added
            totaldollarcostavgcost += monthlyallocation

            #print("$Price:")
            #print(currentprice)
            #print("No. of Stock:")
            #print(numberofstocks)
    print("Monthly DCA")
    print(dca_numberofstocksheld)
    print(totaldollarcostavgcost)

In [23]:
comparetodca(0.1,0.1,800,'NVDA1990_2016.csv') # Money Increment, Percent Deviation, Monthly Allocation, Stock Name

Price Increase
880.0000000000001
15
52.639999
[   0.          0.        800.        789.599985]
Price Increase
868.5599835
33
47.27
[   0.        800.        789.599985  850.86    ]
Price Increase
935.946
58
36.830002
[ 800.        789.599985  850.86      920.75005 ]
53
32.130001
[ 789.599985  850.86      920.75005  -160.650005]
Price Increase
1759.383967
116
27.52
[  850.86       920.75005   -160.650005  1733.76    ]
135
32.939999
[  920.75005   -160.650005  1733.76       625.859981]
Price Increase
968.3079714
170
27.41
[ -160.650005  1733.76       625.859981   959.35    ]
207
22.700001
[ 1733.76       625.859981   959.35       839.900037]
249
19.84
[ 625.859981  959.35      839.900037  833.28    ]
288
21.0
[ 959.35      839.900037  833.28      819.      ]
326
21.379999
[ 839.900037  833.28      819.        812.439962]
362
22.42
[ 833.28      819.        812.439962  807.12    ]
402
19.879999
[ 819.        812.439962  807.12      795.19996 ]
442
19.860001
[ 812.439962  807.12      795.

In [4]:
114311/800

142.88875

In [5]:
monthlyallocation

NameError: name 'monthlyallocation' is not defined

In [ ]:
int(6.45)

In [ ]:
int(6.64)

In [ ]:
int(7.99)

In [ ]:
800*1.1

In [ ]:
800/52